In [17]:
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import re
import pandas as pd

comment_dict={}
article_dict={}

comment_list=[]
article_ids=[]


#날짜에 따른 url
date_urls=[]
dates=[]
month_days= {2:29,3:31, 4:5}
for month in range(4, 3 + 2):
        # 각 월의 일수만큼 반복
        for day in range(1, month_days[month] + 1):
            # 날짜 형식을 맞추어 리스트에 추가
            dates.append(f"2024{month:02d}{day:02d}")
print(dates)


for date in dates:
    url= f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=014&date={date}'
    date_urls.append(url)

headers = {
    'referer': 'https://n.news.naver.com/mnews/article/comment/014/0005157040',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0'
}



cate_article_ids=[]

last_page=None
#페이지에 따른 article_id
for url in date_urls:
    for page in range(1, 50):
        pages_url=url+'&page='+str(page)
        resp= requests.get(pages_url)
        soup= BeautifulSoup(resp.content, 'lxml')
        tags= soup.select('div.list_body.newsflash_body li dl dt a.nclicks\(cnt_flashart\)')
        
        date= pages_url.split('date=')[1].split('&')[0]
        current_page= tags
        if last_page==current_page:
             break
        else:
            last_page=current_page
        
            for tag in tags:
                #카테고리별 기사 id 추가
                article_id=tag['href'].split('/')[-1]
                cate_url= f'https://n.news.naver.com/mnews/article/014/{article_id}'
                resp= requests.get(cate_url)
                soup=BeautifulSoup(resp.content, 'lxml')
                cate_tags= soup.select('em.media_end_categorize_item')
                for cate_tag in cate_tags:
                     if cate_tag.text in ['정치','사회','세계','IT','경제']:

                        cate_article_ids.append(tag['href'].split('/')[-1])

                if tag['href'].split('/')[-1] in cate_article_ids:
                    article_dict[tag['href'].split('/')[-1]]=[date, tag.text.replace('\n','').replace('\t',''), tag['href'], cate_tag.text]
        
        
        

columns = ['기사 날짜','기사 제목', '링크', '댓글', '공감','비공감','댓글 날짜','분류']
data_df=pd.DataFrame(columns=columns)










['20240401', '20240402', '20240403', '20240404', '20240405']


In [6]:
print(len(article_dict))

363


In [19]:
for article_id in article_dict.keys():
    count=0
    url= f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20240326103505&_callback=jQuery3310170325802808738_1711597888495&lang=ko&country=KR&objectId=news014%2C{article_id}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=1&currentPage=1&refresh=true&sort=favorite&includeAllStatus=true&_=1711597888499'
    resp= requests.get(url, headers=headers)
    jquery_len=len(url.split('&')[4].split('=')[1])
    jsn= json.loads(resp.text[jquery_len+1:-2])
    
   
    try:
        first_more_param_next= jsn['result']['morePage']['next']
        comment_count= jsn['result']['count']['comment']

        for i in jsn['result']['commentList']:
            data_df=pd.concat([data_df,pd.DataFrame(data=[[article_dict[article_id][0], article_dict[article_id][1], article_dict[article_id][2],i['contents'], i['sympathyCount'], i['antipathyCount'],i['modTime'][:10],article_dict[article_id][3]]], columns=columns)])
            count+=1
            comment_list.append(i['contents'])
        pages= comment_count//20
        
        
        if count>=20:
        #more_param_next 바꿔야함
            url_first= f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20240319154009&_callback=jQuery33103532086850567042_1711598947128&lang=ko&country=KR&objectId=news014%2C{article_id}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=2&currentPage=1&refresh=false&sort=FAVORITE&current=818804297583034494&prev=818796530168234143&moreParam.direction=next&moreParam.prev=100004700004u067y7595e8om7&moreParam.next={first_more_param_next}&includeAllStatus=true&_=1711598947133'
            for page in range(2, pages+2):
                resp= requests.get(url_first, headers=headers)
                jquery_len=len(url_first.split('&')[4].split('=')[1])


                jsn= json.loads(resp.text[jquery_len+1:-2])
                next_param= jsn['result']['morePage']['next']

                url_first= f'https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=news&templateId=default_politics_m3&pool=cbox5&_cv=20240319154009&_callback=jQuery33103532086850567042_1711598947128&lang=ko&country=KR&objectId=news014%2C{article_id}&categoryId=&pageSize=20&indexSize=10&groupId=&listType=OBJECT&pageType=more&page=2&currentPage=3&refresh=false&sort=FAVORITE&current=818804297583034494&prev=818796530168234143&moreParam.direction=next&moreParam.prev=100004700004u067y7595e8om7&moreParam.next={next_param}&includeAllStatus=true&_=1711598947133'
                

                for i in jsn['result']['commentList']:
                    data_df=pd.concat([data_df,pd.DataFrame(data=[[article_dict[article_id][0], article_dict[article_id][1], article_dict[article_id][2],i['contents'], i['sympathyCount'], i['antipathyCount'],i['modTime'][:10],article_dict[article_id][3]]], columns=columns)])
                    comment_list.append(i['contents'])
    except KeyError:
        continue
                    




In [24]:
len(comment_list)

17469

In [20]:
data_df

,기사 날짜,기사 제목,링크,댓글,공감,비공감,댓글 날짜,분류
0,20240401,"尹 담화에 똘똘 뭉치는 여권 ""대통령 탓 말고 현장 뛰라""",https://n.news.naver.com/mnews/article/014/000...,이게 맞는 여권의 자세고 태도죠. 어디서 굴러먹다 들어온지 몇달도 안된 좌파 찌그래...,90,23,2024-04-01,정치
0,20240401,"尹 담화에 똘똘 뭉치는 여권 ""대통령 탓 말고 현장 뛰라""",https://n.news.naver.com/mnews/article/014/000...,맞습니다.,48,9,2024-04-01,정치
0,20240401,"尹 담화에 똘똘 뭉치는 여권 ""대통령 탓 말고 현장 뛰라""",https://n.news.naver.com/mnews/article/014/000...,"대통령탓하는 여당 후보는 전부 사퇴해라! 자격미달인겋들이 욕심만 디룩디룩, 느그들 ...",37,5,2024-04-01,정치
0,20240401,"尹 담화에 똘똘 뭉치는 여권 ""대통령 탓 말고 현장 뛰라""",https://n.news.naver.com/mnews/article/014/000...,가만보면 유승민이 묻은 인간들이 무능력 할 줄 아는거라곤 등에 칼꽂는것 ㅉㅉ 국민이...,37,6,2024-04-01,정치
0,20240401,"尹 담화에 똘똘 뭉치는 여권 ""대통령 탓 말고 현장 뛰라""",https://n.news.naver.com/mnews/article/014/000...,인기 없는 것들이 개딸 연들 눈에라도 들어볼라고 개 솔이. 왈왈. . 잘들었다. 기...,29,3,2024-04-01,정치
...,...,...,...,...,...,...,...,...
0,20240405,"마잉주, 중국 전설 속 시조 황제릉 제사 참석",https://n.news.naver.com/mnews/article/014/000...,지진나는 이유를 알겠다,0,1,2024-04-05,세계
0,20240405,韓-이태리 ‘尹 G7 참석’ 협의..재무장관회의 초청,https://n.news.naver.com/mnews/article/014/000...,건희 명품쇼핑하러 가고싶을껀데 우짜네,1,1,2024-04-05,정치
0,20240405,韓-이태리 ‘尹 G7 참석’ 협의..재무장관회의 초청,https://n.news.naver.com/mnews/article/014/000...,"독일과 완전 팽 당하고 또 그 짝 나라, 50년 폭망 정부에...\n거니 명품뇌물...",0,0,2024-04-05,정치
0,20240405,韓-이태리 ‘尹 G7 참석’ 협의..재무장관회의 초청,https://n.news.naver.com/mnews/article/014/000...,이태리라 써서 순간 한국이 이태리 타월 수출 한다는줄 알았네 ㅋㅋ,0,0,2024-04-05,정치


In [10]:
len(data_df[data_df['분류']=='경제'])


14320

In [21]:
data_df.to_csv('C:/Users/USER/text_mining_project/text_mining_project/crawling_all_4_1_5.csv')


### 기사 분류용 크롤링

In [106]:
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import re
import pandas as pd

comment_dict={}
article_dict={}
article_name_list=[]
section_list=[]

comment_list=[]
article_ids=[]


#날짜에 따른 url
date_urls=[]
dates=[]
month_days= {2:29,3:10}
for month in range(3, 3 + 1):
        # 각 월의 일수만큼 반복
        for day in range(5, month_days[month] + 1):
            # 날짜 형식을 맞추어 리스트에 추가
            dates.append(f"2024{month:02d}{day:02d}")
print(dates)


for date in dates:
    url= f'https://news.naver.com/main/list.naver?mode=LPOD&mid=sec&oid=014&date={date}'
    date_urls.append(url)

headers = {
    'referer': 'https://n.news.naver.com/mnews/article/comment/014/0005157040',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0'
}
count=0


cate_article_ids=[]

last_page=None
#페이지에 따른 article_id
for url in date_urls:
    for page in range(1, 50):
        pages_url=url+'&page='+str(page)
        resp= requests.get(pages_url)
        soup= BeautifulSoup(resp.content, 'lxml')
        tags= soup.select('div.list_body.newsflash_body li dl dt a.nclicks\(cnt_flashart\)')
        
        date= pages_url.split('date=')[1].split('&')[0]
        current_page= tags
        if last_page==current_page:
             break
        else:
            last_page=current_page
        
            for tag in tags:
                #카테고리별 기사 id 추가
                article_id=tag['href'].split('/')[-1]
                cate_url= f'https://n.news.naver.com/mnews/article/014/{article_id}'
                resp= requests.get(cate_url)
                soup=BeautifulSoup(resp.content, 'lxml')
                cate_tags= soup.select('em.media_end_categorize_item')
                title_tags= soup.select('h2#title_area span')
                
                for cate_tag in cate_tags:
                     if cate_tag.text in ['생활','문화','정치','IT','과학','경제','사회']:
                    
                        cate_article_ids.append(tag['href'].split('/')[-1])

                if tag['href'].split('/')[-1] in cate_article_ids:
                    article_name_list.append(title_tags[0].text)
                    section_list.append(cate_tags[0].text)
                
                    
    
        
        

columns = ['기사 제목', '분류']
data_df=pd.DataFrame({'기사제목':article_name_list,'분류':section_list})









['20240305', '20240306', '20240307', '20240308', '20240309', '20240310']


In [107]:
data_df

,기사제목,분류
0,"오석규 경기도의원, GTX-E, 8호선 공용선로 사용 제안",사회
1,"오석규 경기도의원, GTX-E, 8호선 공용선로 사용 제안",사회
2,"이형섭 국민의힘 예비후보 ""의정부, 이제는 바꿀 때"" [2024 총선]",정치
3,"이형섭 국민의힘 예비후보 ""의정부, 이제는 바꿀 때"" [2024 총선]",정치
4,서울 역세권 골목길 용적률 최고 1100% 상향,경제
...,...,...
4960,[뉴욕증시 주간전망] CPI·PPI 발표에 촉각...숨 고르기 지속 여부도 관건,경제
4961,증시 호황에 미 가계자산 사상최대,경제
4962,증시 호황에 미 가계자산 사상최대,경제
4963,"테슬라, 올해 맥도널드·디즈니 시총만큼 날렸다...2년여 사이 반토막",경제


In [109]:
df_unique= data_df.drop_duplicates()

In [112]:
df_unique.to_csv('C:/Users/USER/text_mining_project/text_mining_project/title_section.csv')